# Function and snippet unit tests and scratch code

In [26]:
# optimization parameter selection
all = randn((1,20))
free_index = Int.(vcat(collect(1:4), [12, 18]))
fixed_index = Int.(setdiff(collect(1:length(all)), free_index))
free = all[free_index]
fixed = all[fixed_index];

In [ ]:
"""
**Required Args**
 - `params`: A vector containing all parameters used in the ODE Model
 - `free_index`: A vector of integer values specifying the indices of free parameters to be fit by the optimization routine
 - `variance_index`: A vector of integer values specifying which of the free variables are variances for MLE estimation. Must be a subset of free_index
 - `ics`: Initial state variable values
 - `lb` and `ub`: Vectors with lower and upper bounds for free parameters (must be same length)
 - `data`: Dataframe with patient lab values

 ### _Note_: a Dataframe named dose_df with patient LT4 dosing MUST be defined or the function will throw an error
 
 **Keyword Args**
 - `model`: defaults to `thyrosimIM!`. Can provide alternate model here
 - `loss_function`: How loss is calculated from data (WLS, MLE, custom function....)
 - `use_callback`: specify true if patient has medication values. These must be provided as a dataframe named `dose_df`
 - `optimization_routine`: Nelder-Mead(), Newton(), LBFGS(), etc.. See [Optim.jl Documentation](https://docs.sciml.ai/Optimization/stable/optimization_packages/optim/) for available search routines
 - `iterations`: maximum iterations to run for. Defaults to 100k iterations
 - `time_limit`: maximum time (in seconds) to run parameter search. Note this applies only to optim and does not account for time taken to actually solve ODE problem and calculate loss. Usually takes about double the time you enter here to complete the search if maximum iterations is not reached
 """
function fit_all(
    params::Vector,
    free_index::Vector{Int},
    variance_index::Vector{Int},
    ics::Vector,
    lb::Vector,
    ub::Vector,
    data::DataFrame;
    model=thyrosimIM!,
    loss_function::Function=demeaned_neg_logl,
    use_callback::Bool = false,
    optimization_routine = NelderMead(),
    iterations::Int = 100000,
    time_limit::Number = 200.0)

    # set fixed parameter indexing and split params vector into free and fixed
    fixed_index = Int.(setdiff(collect(1:length(params)), free_index))
    free_params_initial = Int.(vcat(params[free_index], params[variance_index]))
    fixed_params = params[fixed_index]

    print("Fitting...")
    return optimize(free_params -> objective(free_params, fixed_params, variance_index, ics, lb, ub, data, 
        model=model, free_indicies=free_indicies, loss_function=loss_function, use_callback=use_callback),
        free_params_initial, optimization_routine, 
        Optim.Options(time_limit = time_limit, iterations = iterations, g_tol=1e-5,
        show_trace = false, allow_f_increases=true))
end

function objective(
    free_parameters::Vector,
    fixed_parameters::Vector,
    variance_index::Vector,
    ics::Vector,
    lb::Vector,
    ub::Vector,
    data::DataFrame;
    model=thyrosimIM!,
    loss_function::Function=neg_logl,
    solver=Rosenbrock23(),
    use_callback::Bool = false)

    # Bound solution space - needs to be replaced with Optim native options
    for (i, parameter) in enumerate(free_parameters[1:end-9])
        if !(lb[i] ≤ parameter ≤ ub[i])
            # println("Parameter $i with value $parameter was out of bounds $(lb[i]) -- $(ub[i]).")
            return Inf
        end
    end

    variances = free_parameters[variance_index]

    # assign t from data, initialize thyrosim problem and compute solution
    t = data.t; tspan = (t[1], t[end])

    sol = simulate(model, ics, tspan, free_parameters, fixed_parameters, use_callback = use_callback)

    # For now, we are fitting 6 state variables (1,4,7, 20:22). Thus, last 6 parameters must be
    # corresponding variance of the state variables for MLE, can make variable SV number in future
    loss = loss_function(sol, t, data, variances)

    return loss
end

In [ ]:
function lsq_loss(sol, time, data, _)
    data_columns = names(data)[2:6] # this selects T4,T3, TSH, Lymphocytes, Ab
    sol_indicies = [[1],[4],[7],[20,21,22],[25]] # indicies for T4, T3, TSH, Lymphocytes, Ab
    loss_vector = [0,0,0,0,0] # loss for T4,T3, TSH, Lymphocytes, Ab respectivley

    for (i, column) in enumerate(data_columns)
        if any(!ismissing, data[!, column])
            column_mean = mean(skipmissing(data[!, column]))
            for (j, t) in enumerate(time)
                datapoint = data[!, column][j]
                if ismissing(datapoint)
                    loss_vector[i] = loss_vector[i]
                else
                    predicted = 0
                    if column == "FT4"
                        predicted = FT4(sol(t)[sol_indicies[i]])
                        loss_vector[i]+= ((predicted - datapoint)/column_mean)^2
                    elseif column == "FT3"
                        predicted = FT3(sol(t)[sol_indicies[i]])
                        loss_vector[i]+= ((predicted - datapoint)/column_mean)^2
                    elseif column == "Ab"
                        predicted = TPOConvert(sol(t)[sol_indicies[i]])
                        loss_vector[i]+= ((predicted - datapoint)/column_mean)^2
                    else
                        for state_variable in sol_indicies[i]
                            predicted += sol(t)[state_variable]
                        end
                        loss_vector[i] += ((predicted - datapoint)/column_mean)^2
                    end
                    if j == 1 || j == 12 || j == 24 
                        println("Predicted $column: $predicted, t=$(24*j)")
                        println("Datapoint $column: $datapoint, t=$(24*j)")
                        println("Difference: $(predicted - datapoint), t=$(24*j)")
                    end
                end
            end
        else
            loss_vector[i] += 0
        end
    end
    
    return sum(loss_vector)
end

In [ ]:
function bounded_log_lsq_loss(sol, time, data, bounds)
    data_columns = names(data)[2:6] # this selects T4,T3, TSH, Lymphocytes, Ab
    sol_indicies = [[1],[4],[7],[20,21,22],[25]] # indicies for T4, T3, TSH, Lymphocytes, Ab
    loss_vector = [0.0,0.0,0.0,0.0,0.0] # loss for T4,T3, TSH, Lymphocytes, Ab respectivley
    for (i, column) in enumerate(data_columns)
        if any(!ismissing, data[!, column])
            μ = mean(skipmissing(data[!, column]))
            for (j, t) in enumerate(time)
                datapoint = data[!, column][j]
                if ismissing(datapoint)
                    loss_vector[i] = loss_vector[i]
                else
                    predicted = 0
                    if column == "FT4"
                        predicted = FT4(sol(t)[sol_indicies[i]])
                    elseif column == "FT3"
                        predicted = FT3(sol(t)[sol_indicies[i]])
                    elseif column == "Ab"
                        predicted = TPOConvert(sol(t)[sol_indicies[i]])
                    else
                        for state_variable in sol_indicies[i]
                            predicted += sol(t)[state_variable]
                        end
                    end
                    # if predicted out of bounds, assign large penalty. Else assign log_sql_loss
                    (predicted < bounds[i][1] || predicted > bounds[i][2] ) : (loss_vector[i] += 10e6) : loss_vector[i] += ((log10(predicted^2) - log10(datapoint^2))/log10(μ^2))^2
                end
            end
        else
            loss_vector[i] += 0
        end
    end

    global GLOBAL_LOSS
    GLOBAL_LOSS = vcat(GLOBAL_LOSS, loss_vector')
    
    return loss_vector
end

In [5]:
for (i, value) in enumerate([1,4,7,(20,21,22),25])
    println(i)
    if value isa Int
        println(value)
    else
        for i in value
            print(i)
        end
    end
end

1
1
2
4
3
7
4
2021225
25


In [2]:
g = ()
!isempty(g)

false